In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import math
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import gc

In [ ]:
def count_na(df):
  return df.isna().sum().sum()

In [ ]:
# reading in FASTA files containing the Amino Acid sequences
def read_fasta(file_names):
    """
    Reads a FASTA files and returns a pd DataFrame with the protein names and Amino Acid sequences
    # Params
    file_names: list of fasta file names to read
    
    # Returns
    df: the DataFrame containing the names and sequences from all inputted files
    """
    df = pd.DataFrame()
    names = []
    sequences = []
    ids = []
    db_id = []
    for file_name in file_names:
        seq = ""
        with open(file_name, "r") as f:
            for line in f:
                # if its reading a comment line extract the name of the target
                if line[0] == '>':
                    sequences.append(seq)
                    seq = ""
                    names.append(' '.join(str(v) for v in line.split(' (D')[0].split()[2:]).lower())
                    ids.append(line.split()[1])
                # if its not on a comment line extract the sequence
                else:
                    seq += line
            # adding the last sequence
            sequences.append(seq)
            # removing empty strings added to list
            sequences.remove("")
    df['Name'] = names
    df['uniprot id'] = ids
    df['sequence'] = sequences
    return df

# Drug/Protein Feature Vectors

In [ ]:
# protein/drug descriptors
desc = pd.read_csv('/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/approved_drug_descriptors.csv')
adj = pd.read_csv('/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/protein_adjacency_matrix.csv')

desc.rename(columns={'DrugBank ID':'Drug ID'}, inplace=True)
# lowercasing all drug names
desc['Name'] = desc['Name'].apply(lambda x: x.lower())

**All SMILES in `desc` are** *Canonical SMILES*

In [ ]:
desc

,Drug ID,Name,SMILES,ALogPS_logP,ALogPS_logS,nA:(CDK2),nR:(CDK2),nN:(CDK2),nD:(CDK2),nC:(CDK2),nF:(CDK2),nQ:(CDK2),nE:(CDK2),nG:(CDK2),nH:(CDK2),nI:(CDK2),nP:(CDK2),nL:(CDK2),nK:(CDK2),nM:(CDK2),nS:(CDK2),nT:(CDK2),nY:(CDK2),nV:(CDK2),nW:(CDK2),nAcid:(CDK2),ALogP:(CDK2),ALogp2:(CDK2),AMR:(CDK2),apol:(CDK2),naAromAtom:(CDK2),nAromBond:(CDK2),nAtom:(CDK2),ATSc1:(CDK2),ATSc2:(CDK2),ATSc3:(CDK2),ATSc4:(CDK2),ATSc5:(CDK2),ATSm1:(CDK2),ATSm2:(CDK2),...,SYMC2Z:(Mersy),SYMC2:(Mersy),SYMC3X:(Mersy),SYMC3Y:(Mersy),SYMC3Z:(Mersy),SYMC3:(Mersy),SYMC4X:(Mersy),SYMC4Y:(Mersy),SYMC4Z:(Mersy),SYMC4:(Mersy),SYMC5X:(Mersy),SYMC5Y:(Mersy),SYMC5Z:(Mersy),SYMC5:(Mersy),SYMC6X:(Mersy),SYMC6Y:(Mersy),SYMC6Z:(Mersy),SYMC6:(Mersy),SYMS1X:(Mersy),SYMS1Y:(Mersy),SYMS1Z:(Mersy),SYMS1:(Mersy),SYMS2:(Mersy),SYMS3X:(Mersy),SYMS3Y:(Mersy),SYMS3Z:(Mersy),SYMS3:(Mersy),SYMS4X:(Mersy),SYMS4Y:(Mersy),SYMS4Z:(Mersy),SYMS4:(Mersy),SYMS5X:(Mersy),SYMS5Y:(Mersy),SYMS5Z:(Mersy),SYMS5:(Mersy),SYMS6X:(Mersy),SYMS6Y:(Mersy),SYMS6Z:(Mersy),SYMS6:(Mersy),CHIR:(Mersy)
0,DB00007,leuprolide,CCNC(=O)[C@@H]1CCCN1C(=O)[C@@H](NC(=O)[C@@H](N...,1.04,-4.55,9.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,9.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-4.26,18.200,296.0,187.0,20.0,21.0,171.0,2.270,-1.0900,-0.05810,0.828000,-1.500000,102.0,101.0,...,0.379,0.385,0.294,0.254,0.427,0.329,0.312,0.264,0.420,0.335,0.292,0.257,0.423,0.328,0.299,0.261,0.434,0.336,0.288,0.464,0.342,0.369,0.233,0.195,0.243,0.399,0.284,0.223,0.257,0.406,0.300,0.204,0.260,0.402,0.294,0.214,0.260,0.401,0.297,0.652
1,DB00014,goserelin,OC[C@@H](C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](...,0.30,-4.65,9.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,9.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,-5.59,31.200,302.0,191.0,20.0,21.0,175.0,2.640,-1.2500,-0.16200,0.922000,-1.480000,108.0,107.0,...,0.377,0.420,0.247,0.276,0.359,0.295,0.279,0.308,0.361,0.317,0.275,0.285,0.363,0.308,0.285,0.301,0.373,0.321,0.395,0.376,0.363,0.378,0.219,0.212,0.237,0.314,0.256,0.258,0.256,0.333,0.283,0.247,0.246,0.339,0.279,0.255,0.260,0.340,0.286,0.648
2,DB00035,desmopressin,NC(=O)C[C@@H]1NC(=O)[C@H](CCC(=O)N)NC(=O)[C@H]...,-1.01,-3.99,7.0,1.0,1.0,0.0,1.0,4.0,1.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-5.53,30.600,246.0,154.0,12.0,12.0,138.0,2.150,-0.9620,-0.09660,0.700000,-1.330000,101.0,94.4,...,0.385,0.380,0.267,0.242,0.395,0.305,0.276,0.285,0.373,0.313,0.281,0.295,0.381,0.321,0.289,0.295,0.386,0.325,0.342,0.349,0.317,0.336,0.228,0.270,0.310,0.325,0.302,0.269,0.309,0.311,0.297,0.272,0.314,0.305,0.297,0.272,0.308,0.309,0.296,0.670
3,DB00091,cyclosporine,C/C=C/C[C@H]([C@H]([C@H]1C(=O)N[C@@H](CC)C(=O)...,4.12,-5.09,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,-3.12,9.740,319.0,205.0,0.0,0.0,196.0,2.260,-1.2400,-0.05210,1.370000,-2.260000,98.3,93.8,...,0.470,0.491,0.453,0.529,0.355,0.450,0.461,0.534,0.388,0.464,0.467,0.512,0.383,0.456,0.476,0.548,0.396,0.477,0.359,0.419,0.557,0.452,0.472,0.460,0.525,0.342,0.447,0.454,0.510,0.370,0.448,0.453,0.503,0.374,0.446,0.454,0.529,0.388,0.460,0.489
4,DB00104,octreotide,NCCCC[C@@H]1NC(=O)[C@H](NC(=O)[C@H](Cc2ccccc2)...,0.42,-4.93,7.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-2.61,6.790,259.0,155.0,21.0,22.0,137.0,1.700,-0.6400,-0.30200,0.586000,-0.923000,94.6,90.8,...,0.406,0.377,0.348,0.305,0.358,0.338,0.354,0.336,0.367,0.353,0.322,0.331,0.374,0.342,0.343,0.331,0.373,0.349,0.343,0.287,0.334,0.322,0.363,0.378,0.373,0.350,0.367,0.362,0.344,0.337,0.348,0.350,0.364,0.323,0.346,0.362,0.353,0.326,0.347,0.610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,DB15598,ferric maltol,O=c1ccoc(c1[O-])C.O=c1ccoc(c1[O-])C.O=c1ccoc(c...,-0.24,0.03,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
import glob

# reading the fasta files with the target amino acid sequences
l = [filename for filename in glob.glob("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/target_structures/*.fasta")]
seq_df = read_fasta(l)
seq_df.drop_duplicates(inplace=True)
seq_df.sequence = seq_df.sequence.apply(lambda s: s.replace('\n', ''))

seq_df.rename(columns={'uniprot id':"UniProt ID"}, inplace=True)
# removing proteins that are not in the adj dataframe; these are proteins that we don't have domain data for
seq_df = seq_df.loc[seq_df['UniProt ID'].isin(adj['UniProt ID'])]
# removing duplicates
seq_df.drop_duplicates(inplace=True)

seq_df.head()

,Name,UniProt ID,sequence
0,"glutaminase liver isoform, mitochondrial",Q9UI32,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...
1,coagulation factor xiii a chain,P00488,MSETSRTAFGGRRAVPPNNSNAAEDDLPTVELQGVVPRGVNLQEFL...
2,"nitric oxide synthase, inducible",P35228,MACPWKFLFKTKFHQYAMNGEKDINNNVEKAPCATSSPVTQDDLQY...
3,alcohol dehydrogenase class-3,P11766,MANEVIKCKAAVAWEAGKPLSIEEIEVAPPKAHEVRIKIIATAVCH...
4,"aminomethyltransferase, mitochondrial",P48728,MQRAVSVVARLGFRLQAFPPALCRPLSCAQEVLRRTPLYDFHLAHG...


# DrugBank Data -- Positive DTIs
Loading the postive (true) DTIs dowloaded from the DrugBank website. Labeling all of them as `1`.

In [ ]:
import glob

# reading all csv files in the drugbank_DTIs folder to create DataFrame with all drugbank dtis
l = [pd.read_csv(filename) for filename in glob.glob("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/drugbank_DTIs/*.csv")]
drugbank_dtis = pd.concat(l, axis=0).assign(label=1).rename(columns={'DrugBank ID':'Drug ID'})
drugbank_dtis['Name'] = drugbank_dtis['Name'].apply(lambda s: s.lower())

drugbank_dtis.head()

,Drug ID,Name,Type,UniProt ID,UniProt Name,label
0,DB00006,bivalirudin,SmallMoleculeDrug,P05164,Myeloperoxidase,1
1,DB00008,peginterferon alfa-2a,BiotechDrug,P05177,Cytochrome P450 1A2,1
2,DB00011,interferon alfa-n1,BiotechDrug,P05177,Cytochrome P450 1A2,1
3,DB00013,urokinase,BiotechDrug,P39900,Macrophage metalloelastase,1
4,DB00018,interferon alfa-n3,BiotechDrug,P05177,Cytochrome P450 1A2,1


In [ ]:
def read_smi2(file_path):
    ids = []
    smi = []
    name = []
    with open(file_path, "r") as f:
        for line in f:
            line = line.replace('\n', '').split(';')
            name.append(line[1].lower())
            line[0] = line[0].split('\t')
            smi.append(line[0][0])
            ids.append(line[0][1])
    return pd.DataFrame(data={'Drug ID':ids, "Name":name, "SMILES":smi})

In [ ]:
drugbank_smi_df = read_smi2('/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/approved_drug.can')

In [ ]:
# adding SMILES to drugbank dtis
drugbank_dtis = drugbank_dtis.merge(drugbank_smi_df.drop(columns=['Name']), how='inner', on=['Drug ID'])

In [ ]:
# removing drugs that are not in the descriptors dataframe (by SMILES)
df = drugbank_dtis.loc[drugbank_dtis['SMILES'].isin(desc['SMILES'])]
print(f"Removing {len(set(drugbank_dtis['SMILES'])) - len(set(df['SMILES']))} drugs that are not in the descriptors dataframe")
drugbank_dtis = df
del df

# removing drugs from descriptors dataframe that are not in the drugbank_dtis dataframe
df = desc.loc[desc['SMILES'].isin(drugbank_dtis['SMILES'])]
print(f"Removing {len(set(desc['SMILES'])) - len(set(df['SMILES']))} drugs that are not in the drugbank_dtis dataframe")
desc = df
del df

Removing 101 drugs that are not in the descriptors dataframe
Removing 427 drugs that are not in the drugbank_dtis dataframe


In [ ]:
# dropping duplicates in the dataframe
drugbank_dtis.drop_duplicates(inplace=True)
drugbank_dtis

,Drug ID,Name,Type,UniProt ID,UniProt Name,label,SMILES
2,DB00035,desmopressin,SmallMoleculeDrug,P23219,Prostaglandin G/H synthase 1,1,NC(=O)C[C@@H]1NC(=O)[C@H](CCC(=O)N)NC(=O)[C@H]...
3,DB00035,desmopressin,SmallMoleculeDrug,P35354,Prostaglandin G/H synthase 2,1,NC(=O)C[C@@H]1NC(=O)[C@H](CCC(=O)N)NC(=O)[C@H]...
4,DB00035,desmopressin,SmallMoleculeDrug,P30518,Vasopressin V2 receptor,1,NC(=O)C[C@@H]1NC(=O)[C@H](CCC(=O)N)NC(=O)[C@H]...
5,DB00035,desmopressin,SmallMoleculeDrug,P37288,Vasopressin V1a receptor,1,NC(=O)C[C@@H]1NC(=O)[C@H](CCC(=O)N)NC(=O)[C@H]...
6,DB00035,desmopressin,SmallMoleculeDrug,P47901,Vasopressin V1b receptor,1,NC(=O)C[C@@H]1NC(=O)[C@H](CCC(=O)N)NC(=O)[C@H]...
...,...,...,...,...,...,...,...
17920,DB14507,lithium citrate,SmallMoleculeDrug,P42263,Glutamate receptor 3,1,[O-]C(=O)C(CC(=O)[O-])(CC(=O)[O-])O.[Li+].[Li+...
17921,DB14783,diroximel fumarate,SmallMoleculeDrug,Q9GZZ6,Neuronal acetylcholine receptor subunit alpha-10,1,COC(=O)/C=C/C(=O)OCCN1C(=O)CCC1=O
17922,DB14914,flortaucipir f-18,SmallMoleculeDrug,P10636,Microtubule-associated protein tau,1,[18F]c1ccc(cn1)c1ccc2c(c1)[nH]c1c2cncc1
17923,DB14914,flortaucipir f-18,SmallMoleculeDrug,P21397,Amine oxidase [flavin-containing] A,1,[18F]c1ccc(cn1)c1ccc2c(c1)[nH]c1c2cncc1


# Negative DTIs
Randomly selecting `n` DTIs from all possible drugbank DTIs where `n` is the number of positive DTIs

## Run this

In [ ]:
def read_smi(file_path):
  """
  Read an .smi or .can file and store the ids and SMILES in a DataFrame
  """
  smi = []
  ids = []
  with open(file_path, "r") as f:
      for line in f:
          line = line.split('\t')
          ids.append(line[1].replace('\n', ''))
          smi.append(line[0])
  return pd.DataFrame(data={"Drug ID":ids, "SMILES":smi})

In [ ]:
# finding all possible DTIs
neg_dtis = []
drugs = desc['SMILES']
targets = adj['UniProt ID']

for d in tqdm(drugs):
  for t in targets:
    neg_dtis.append((d, t))

# creating DataFrame from the computed DTIs
neg_dtis_df = pd.DataFrame()
neg_dtis_df['SMILES'] = [i[0] for i in neg_dtis]
neg_dtis_df['UniProt ID'] = [i[1] for i in neg_dtis]
# removing dtis that are in the positive dti set
neg_dtis_df = pd.concat([neg_dtis_df, drugbank_dtis[['SMILES', 'UniProt ID']]]).drop_duplicates(keep=False)
neg_dtis_df.head()

# randomly sampling an equal amount to the number of positive DTIs
neg_dti_sample = neg_dtis_df.sample(n=drugbank_dtis.shape[0]).assign(label=0)
neg_dti_sample = neg_dti_sample.merge(drugbank_smi_df[['SMILES', 'Drug ID']], how='inner', on=['SMILES'])

neg_dti_sample

,SMILES,UniProt ID,label,Drug ID
0,CN([C@@H]1C(=C(C(=O)N)C(=O)[C@@]2([C@H]1[C@@H]...,Q13085,0,DB00254
1,CN([C@@H]1C(=C(C(=O)N)C(=O)[C@@]2([C@H]1[C@@H]...,Q9NSE4,0,DB00254
2,CN([C@@H]1C(=C(C(=O)N)C(=O)[C@@]2([C@H]1[C@@H]...,P47871,0,DB00254
3,CN([C@@H]1C(=C(C(=O)N)C(=O)[C@@]2([C@H]1[C@@H]...,O00329,0,DB00254
4,CN([C@@H]1C(=C(C(=O)N)C(=O)[C@@]2([C@H]1[C@@H]...,P50870,0,DB00254
...,...,...,...,...
16635,CN(CCc1c[nH]c2c1cc(C[C@H]1COC(=O)N1)cc2)C,P10275,0,DB00315
16636,CN(CCc1c[nH]c2c1cc(C[C@H]1COC(=O)N1)cc2)C,P62745,0,DB00315
16637,OCC(=O)[C@@]12OC(O[C@@H]1C[C@@H]1[C@]2(C)C[C@H...,P40406,0,DB01260
16638,Clc1ccc2c(c1NC1=NCCN1)nsn2,Q96AZ6,0,DB00697


In [ ]:
print(f"Created {neg_dtis_df.shape[0]} negative DTIs that were not in the positive dtis. Randomly sampled {neg_dti_sample.shape[0]} of them for use in machine learning model.")

Created 9655565 negative DTIs that were not in the positive dtis. Randomly sampled 16640 of them for use in machine learning model.


# Test/Training Set
Putting together the positive and negative DTIs and splitting it into testing and training datasets. The test data consists of DTIs with proteins (targets) that are not in the training set (any protein in the test set will be removed from the training set).

## Splitting into Test and Train (RUN this)

In [ ]:
# removing useless columns
drugbank_dtis.drop(columns=['Name', 'Type', 'UniProt Name'], inplace=True)

# combining positive and negative DTIs
dtis = pd.concat([drugbank_dtis, neg_dti_sample])

# removing DTIs whose proteins are not in the sequence DataFrame
dtis = dtis.loc[dtis['UniProt ID'].isin(seq_df['UniProt ID'])]

In [ ]:
dtis.reset_index(drop=True, inplace=True)

In [ ]:
# adding drug descriptors before splitting data
all_desc = pd.DataFrame(desc).drop(columns=['Name'])
dtis = dtis.merge(all_desc, how='inner', on=['SMILES', 'Drug ID'])

In [ ]:
train_dataset, test_dataset = train_test_split(dtis, test_size=0.3, shuffle=True, stratify=dtis['label'])

In [ ]:
train_dataset.reset_index(drop=True, inplace=True)
test_dataset.reset_index(drop=True, inplace=True)

In [ ]:
test_dataset.shape

(9981, 11472)

In [ ]:
del drugbank_dtis, neg_dti_sample, dtis
gc.collect()

12

In [ ]:
train_dataset

,Drug ID,UniProt ID,label,SMILES,ALogPS_logP,ALogPS_logS,nA:(CDK2),nR:(CDK2),nN:(CDK2),nD:(CDK2),nC:(CDK2),nF:(CDK2),nQ:(CDK2),nE:(CDK2),nG:(CDK2),nH:(CDK2),nI:(CDK2),nP:(CDK2),nL:(CDK2),nK:(CDK2),nM:(CDK2),nS:(CDK2),nT:(CDK2),nY:(CDK2),nV:(CDK2),nW:(CDK2),nAcid:(CDK2),ALogP:(CDK2),ALogp2:(CDK2),AMR:(CDK2),apol:(CDK2),naAromAtom:(CDK2),nAromBond:(CDK2),nAtom:(CDK2),ATSc1:(CDK2),ATSc2:(CDK2),ATSc3:(CDK2),ATSc4:(CDK2),ATSc5:(CDK2),ATSm1:(CDK2),...,SYMC2Z:(Mersy),SYMC2:(Mersy),SYMC3X:(Mersy),SYMC3Y:(Mersy),SYMC3Z:(Mersy),SYMC3:(Mersy),SYMC4X:(Mersy),SYMC4Y:(Mersy),SYMC4Z:(Mersy),SYMC4:(Mersy),SYMC5X:(Mersy),SYMC5Y:(Mersy),SYMC5Z:(Mersy),SYMC5:(Mersy),SYMC6X:(Mersy),SYMC6Y:(Mersy),SYMC6Z:(Mersy),SYMC6:(Mersy),SYMS1X:(Mersy),SYMS1Y:(Mersy),SYMS1Z:(Mersy),SYMS1:(Mersy),SYMS2:(Mersy),SYMS3X:(Mersy),SYMS3Y:(Mersy),SYMS3Z:(Mersy),SYMS3:(Mersy),SYMS4X:(Mersy),SYMS4Y:(Mersy),SYMS4Z:(Mersy),SYMS4:(Mersy),SYMS5X:(Mersy),SYMS5Y:(Mersy),SYMS5Z:(Mersy),SYMS5:(Mersy),SYMS6X:(Mersy),SYMS6Y:(Mersy),SYMS6Z:(Mersy),SYMS6:(Mersy),CHIR:(Mersy)
0,DB01126,P39593,0,O=C1C=C[C@]2([C@H](N1)CC[C@@H]1[C@@H]2CC[C@]2(...,5.45,-5.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.550,12.6000,121.0,74.70,6.0,6.0,67.0,0.859,-0.532,0.17300,-0.0611,0.116000,48.30,...,0.590,0.649,0.468,0.492,0.588,0.519,0.499,0.521,0.603,0.543,0.499,0.499,0.610,0.539,0.504,0.503,0.611,0.542,0.597,0.617,0.543,0.587,0.422,0.417,0.449,0.577,0.486,0.449,0.477,0.577,0.504,0.450,0.473,0.581,0.505,0.454,0.464,0.577,0.502,0.419
1,DB14487,P13647,1,[O-]C(=O)C.[O-]C(=O)C.[Zn+2],-0.12,0.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.230,0.0529,12.6,7.79,0.0,0.0,8.0,0.195,-0.091,-0.00629,0.0000,0.000000,5.55,...,0.825,1.000,0.765,0.714,0.744,0.742,0.757,0.719,0.766,0.748,0.748,0.698,0.764,0.738,0.756,0.718,0.775,0.751,1.000,0.793,0.705,1.000,0.681,0.765,0.701,0.679,0.717,0.757,0.717,0.689,0.722,0.748,0.688,0.673,0.705,0.756,0.703,0.691,0.718,0.000
2,DB01283,Q15274,0,OC(=O)Cc1cc(C)ccc1Nc1c(F)cccc1Cl,4.52,-4.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.760,22.6000,75.7,40.50,12.0,12.0,33.0,0.335,-0.124,-0.06690,0.0181,-0.045500,31.10,...,0.493,0.535,0.501,0.491,0.496,0.496,0.490,0.504,0.501,0.499,0.503,0.500,0.503,0.502,0.499,0.503,0.508,0.503,0.569,0.487,0.452,0.505,0.384,0.519,0.450,0.502,0.491,0.480,0.472,0.515,0.490,0.481,0.468,0.500,0.483,0.484,0.462,0.495,0.481,0.485
3,DB00138,Q9P286,0,N[C@H](C(=O)O)CSSC[C@@H](C(=O)O)N,-3.16,-1.16,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.090,1.1900,54.5,29.80,0.0,0.0,26.0,0.519,-0.156,-0.16500,-0.0342,0.071600,30.10,...,0.721,0.730,0.581,0.578,0.685,0.618,0.604,0.595,0.699,0.635,0.603,0.605,0.696,0.637,0.606,0.602,0.696,0.637,0.689,0.640,0.724,0.687,0.627,0.568,0.564,0.714,0.622,0.590,0.578,0.712,0.632,0.596,0.590,0.699,0.632,0.596,0.593,0.701,0.634,0.295
4,DB01054,P9WIE5,0,CCOC(=O)C1=C(C)NC(=C(C1c1cccc(c1)[N+](=O)[O-])...,2.69,-4.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.920,3.7000,91.4,52.00,6.0,6.0,46.0,0.649,-0.384,0.20600,-0.2350,0.279000,31.40,...,0.432,0.525,0.486,0.546,0.468,0.501,0.512,0.507,0.439,0.487,0.513,0.521,0.457,0.498,0.518,0.518,0.462,0.500,0.443,0.433,0.522,0.468,0.346,0.547,0.512,0.454,0.506,0.553,0.501,0.436,0.499,0.553,0.512,0.459,0.509,0.546,0.499,0.452,0.501,0.480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23282,DB00750,Q8NEY4,0,CCCNC(C(=O)Nc1ccccc1C)C,1.86,-2.83,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.906,0.8220,63.9,39.20,6.0,6.0,36.0,0.234,-0.116,-0.05080,0.0915,-0.033900,17.50,...,0.501,0.5

## Adding Features to each Dataset
Adding the protein and drug descriptors to each DTI in the dataset

### Protein Descriptors

In [ ]:
# total number of data points
train_dataset.shape[0] + test_dataset.shape[0]

33268

In [ ]:
# total number of drug features + other info
train_dataset.shape[1]

11471

#### ACC, DC, TC

In [ ]:
from joblib import dump
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# reading in FASTA files containing the Amino Acid sequences
def read_fasta(file_names):
    """
    Reads a FASTA files and returns a pd DataFrame with the protein names and Amino Acid sequences
    # Params
    file_names: list of fasta file names to read
    
    # Returns
    df: the DataFrame containing the names and sequences from all inputted files
    """
    df = pd.DataFrame()
    names = []
    sequences = []
    ids = []
    db_id = []
    for file_name in file_names:
        seq = ""
        with open(file_name, "r") as f:
            for line in f:
                # if its reading a comment line extract the name of the target
                if line[0] == '>':
                    sequences.append(seq)
                    seq = ""
                    names.append(' '.join(str(v) for v in line.split(' (D')[0].split()[2:]).lower())
                    ids.append(line.split()[1])
                # if its not on a comment line extract the sequence
                else:
                    seq += line
            # adding the last sequence
            sequences.append(seq)
            # removing empty strings added to list
            sequences.remove("")
    df['Name'] = names
    df['uniprot id'] = ids
    df['sequence'] = sequences
    return df

In [ ]:
import glob

# reading the fasta files with the target amino acid sequences
l = [filename for filename in glob.glob("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/target_structures/*.fasta")]
seq_df = read_fasta(l)
seq_df.drop_duplicates(inplace=True)
seq_df.sequence = seq_df.sequence.apply(lambda s: s.replace('\n', ''))

seq_df.rename(columns={'uniprot id':"UniProt ID"}, inplace=True)
# removing proteins that are not in the adj dataframe; these are proteins that we don't have domain data for
seq_df = seq_df.loc[seq_df['UniProt ID'].isin(adj['UniProt ID'])]
# removing duplicates
seq_df.drop_duplicates(inplace=True)

seq_df.head()

,Name,UniProt ID,sequence
0,"glutaminase liver isoform, mitochondrial",Q9UI32,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...
1,coagulation factor xiii a chain,P00488,MSETSRTAFGGRRAVPPNNSNAAEDDLPTVELQGVVPRGVNLQEFL...
2,"nitric oxide synthase, inducible",P35228,MACPWKFLFKTKFHQYAMNGEKDINNNVEKAPCATSSPVTQDDLQY...
3,alcohol dehydrogenase class-3,P11766,MANEVIKCKAAVAWEAGKPLSIEEIEVAPPKAHEVRIKIIATAVCH...
4,"aminomethyltransferase, mitochondrial",P48728,MQRAVSVVARLGFRLQAFPPALCRPLSCAQEVLRRTPLYDFHLAHG...


In [ ]:
# adding sequences to DTIs
train_dataset = train_dataset.merge(seq_df.drop(columns=['Name']), how='inner', on='UniProt ID')
test_dataset = test_dataset.merge(seq_df.drop(columns=['Name']), how='inner', on='UniProt ID')

In [ ]:
# for ACC
acc_vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='char')

acc_train = acc_vectorizer.fit_transform(train_dataset['sequence'])
acc_test = acc_vectorizer.transform(test_dataset['sequence'])

# Creates pandas dataframe of amino acid counts.
# value at index [x, y] is the frequency of feature y in protein x
acc_train = pd.DataFrame.sparse.from_spmatrix(acc_train)
acc_test = pd.DataFrame.sparse.from_spmatrix(acc_test)

# saving ACC vectorizer
dump(acc_vectorizer, '/gdrive/My Drive/Colab Notebooks/REHS/models/acc_vectorizer.joblib')

['/gdrive/My Drive/Colab Notebooks/REHS/models/acc_vectorizer.joblib']

In [ ]:
# for DC
dc_vectorizer = CountVectorizer(ngram_range=(2, 2), analyzer='char')

dc_train = dc_vectorizer.fit_transform(train_dataset['sequence'])
dc_test = dc_vectorizer.transform(test_dataset['sequence'])

# Creates pandas dataframe of amino acid counts.
# value at index [x, y] is the frequency of feature y in protein x
dc_train = pd.DataFrame.sparse.from_spmatrix(dc_train)
dc_test = pd.DataFrame.sparse.from_spmatrix(dc_test)

# saving DC vectorizer
dump(dc_vectorizer, '/gdrive/My Drive/Colab Notebooks/REHS/models/dc_vectorizer.joblib')

['/gdrive/My Drive/Colab Notebooks/REHS/models/dc_vectorizer.joblib']

In [ ]:
# for TC
tc_vectorizer = CountVectorizer(ngram_range=(3, 3), analyzer='char')

tc_train = tc_vectorizer.fit_transform(train_dataset['sequence'])
tc_test = tc_vectorizer.transform(test_dataset['sequence'])

# Creates pandas dataframe of amino acid counts.
# value at index [x, y] is the frequency of feature y in protein x
tc_train = pd.DataFrame.sparse.from_spmatrix(tc_train)
tc_test = pd.DataFrame.sparse.from_spmatrix(tc_test)

# saving ACC vectorizer
dump(tc_vectorizer, '/gdrive/My Drive/Colab Notebooks/REHS/models/tc_vectorizer.joblib')

['/gdrive/My Drive/Colab Notebooks/REHS/models/tc_vectorizer.joblib']

In [ ]:
test_dataset.drop(columns=['sequence'], inplace=True)
train_dataset.drop(columns=['sequence'], inplace=True)

In [ ]:
train_protein_features = pd.concat([train_dataset['UniProt ID'], acc_train, dc_train, tc_train], axis=1, ignore_index=True).rename(columns={0:'UniProt ID'})
test_protein_features = pd.concat([test_dataset['UniProt ID'], acc_test, dc_test, tc_test], axis=1, ignore_index=True).rename(columns={0:'UniProt ID'})

The protein descriptors are not being standard scaled because they are *counts* of substrings of amino acids. Because of this, the numbers, like 0, each have an important meaning (like 0 representing the absensce of that substring in the sequence). Scaling them will not provide any benefits and may cause the numbers to lose their physical meaning.

In [ ]:
# adding domain information to datasets
train_protein_features = train_protein_features.merge(adj, how='inner', on=['UniProt ID'])
test_protein_features = test_protein_features.merge(adj, how='inner', on=['UniProt ID'])

In [ ]:
train_protein_features.drop(columns=['UniProt ID'], inplace=True)
test_protein_features.drop(columns=['UniProt ID'], inplace=True)

In [ ]:
train_protein_features.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,Q9NY64,Q8N1Q1,Q99835,P49674,Q9BQB6,P78508,A3CMA7,Q14032,Q88016,P03765,Q13639,Q6PHW0,P17302,P50747,O08498,P30838,P05655,P13551,Q2QJL3,P52647,O31465,Q8N5Z0,O75380,Q13564,Q6NUS8,P22234,P20020,Q16853,Q9XB61,Q8IKL4,P11233,P95780,Q07817,P14090,Q9RHZ6,Q9NNW7,P0AEG4,P04070,P50914,Q9BY07
0,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
del acc_train, acc_test, dc_train, dc_test, tc_train, tc_test, desc, adj, acc_vectorizer, dc_vectorizer, tc_vectorizer, seq_df
gc.collect()

12

Reducing the number of protein descriptors

In [ ]:
# removing descriptors that are constant
prot_var = VarianceThreshold(threshold=0)
train_protein_features = pd.DataFrame(prot_var.fit_transform(train_protein_features))
test_protein_features = pd.DataFrame(prot_var.transform(test_protein_features))
print(f"{train_protein_features.shape[1]} features left")

13499 features left


In [ ]:
train_protein_features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,13459,13460,13461,13462,13463,13464,13465,13466,13467,13468,13469,13470,13471,13472,13473,13474,13475,13476,13477,13478,13479,13480,13481,13482,13483,13484,13485,13486,13487,13488,13489,13490,13491,13492,13493,13494,13495,13496,13497,13498
0,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,41,8,23,33,16,37,14,15,22,74,12,17,30,34,42,31,18,0,37,5,0,14,5,1,1,3,2,3,1,2,4,8,0,1,1,3,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dump(prot_var, '/gdrive/My Drive/Colab Notebooks/REHS/models/prot_var.joblib')

['/gdrive/My Drive/Colab Notebooks/REHS/models/prot_var.joblib']

In [ ]:
train_dataset.to_csv("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/datasets/train_dataset.csv", index=False)
print("Wrote train data. Now writing test data.")
test_dataset.to_csv("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/datasets/test_dataset.csv", index=False)
print("Wrote test data. Now writing protein data.")
train_protein_features.to_csv("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/datasets/train_protein_features.csv", index=False)
test_protein_features.to_csv("/gdrive/My Drive/Colab Notebooks/REHS/drug_target_data/datasets/test_protein_features.csv", index=False)

Wrote train data. Now writing test data.
Wrote test data. Now writing protein data.
